<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Analyzing Chipotle Data

_Author: Joseph Nelson (DC)_

---

For Project 2, you will complete a series of exercises exploring [order data from Chipotle](https://github.com/TheUpshot/chipotle), compliments of _The New York Times'_ "The Upshot."

For these exercises, you will conduct basic exploratory data analysis (Pandas not required) to understand the essentials of Chipotle's order data: how many orders are being made, the average price per order, how many different ingredients are used, etc. These allow you to practice business analysis skills while also becoming comfortable with Python.

---

## Basic Level

### Part 1: Read in the file with `csv.reader()` and store it in an object called `file_nested_list`.

Hint: This is a TSV (tab-separated value) file, and `csv.reader()` needs to be told [how to handle it](https://docs.python.org/2/library/csv.html).

In [6]:
import pandas as pd

file_nested_list = pd.read_csv('./data/chipotle.tsv',sep='\t')
file_nested_list.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


### Part 2: Separate `file_nested_list` into the `header` and the `data`.


In [16]:
header = file_nested_list.columns
header

Index([u'order_id', u'quantity', u'item_name', u'choice_description',
       u'item_price'],
      dtype='object')

In [15]:
data = file_nested_list.values
data

array([[1L, 1L, 'Chips and Fresh Tomato Salsa', nan, '$2.39 '],
       [1L, 1L, 'Izze', '[Clementine]', '$3.39 '],
       [1L, 1L, 'Nantucket Nectar', '[Apple]', '$3.39 '],
       ..., 
       [1834L, 1L, 'Chicken Salad Bowl',
        '[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]',
        '$11.25 '],
       [1834L, 1L, 'Chicken Salad Bowl',
        '[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]', '$8.75 '],
       [1834L, 1L, 'Chicken Salad Bowl',
        '[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Lettuce]]',
        '$8.75 ']], dtype=object)

---

## Intermediate Level

### Part 3: Calculate the average price of an order.

Hint: Examine the data to see if the `quantity` column is relevant to this calculation.

Hint: Think carefully about the simplest way to do this!

In [247]:
raw_data = file_nested_list
raw_data.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [64]:
total_orders = len(set(raw_data['order_id'])) # get the count of the set of order_id
total_price = raw_data['item_price'].apply(lambda x: float(x[1:])).sum()
ave_price_per_order = total_price/total_orders
print ave_price_per_order

18.8114285714


In [66]:
# by the way average price per unit - just for curiosity
total_quant = raw_data['quantity'].sum()
total_price = raw_data['item_price'].apply(lambda x: float(x[1:])).sum()
ave_price_per_quant = total_price/total_quant
print ave_price_per_quant

6.93888978278


### Part 4: Create a list (or set) named `unique_sodas` containing all of unique sodas and soft drinks that Chipotle sells.

Note: Just look for `'Canned Soda'` and `'Canned Soft Drink'`, and ignore other drinks like `'Izze'`.

In [248]:
# get the choice_description if text'Canned' is in the item_name, strip out the []
unique_sodas = {item[3][1:-1] for item in raw_data.values if 'Canned' in item[2]}  
unique_sodas

{'Coca Cola',
 'Coke',
 'Diet Coke',
 'Diet Dr. Pepper',
 'Dr. Pepper',
 'Lemonade',
 'Mountain Dew',
 'Nestea',
 'Sprite'}

---

## Advanced Level


### Part 5: Calculate the average number of toppings per burrito.

Note: Let's ignore the `quantity` column to simplify this task.

Hint: Think carefully about the easiest way to count the number of toppings!


In [251]:
burritos = raw_data[raw_data['item_name'].str.contains("Burrito")] # get a dataframe that contains Burrito
total_topings_for_burrito = sum([str(item).count(',') + 1 for item in burritos['choice_description']]) # get the count 
ave_toppings = float(total_topings_for_burrito)/len(burritos)
print 'Total burrios:',len(burritos)
print 'Total topping for burrios:',total_topings_for_burrito
print 'Average topping per burrio:',ave_toppings

Total burrios: 1172
Total topping for burrios: 6323
Average topping per burrio: 5.39505119454


### Part 6: Create a dictionary. Let the keys represent chip orders and the values represent the total number of orders.

Expected output: `{'Chips and Roasted Chili-Corn Salsa': 18, ... }`

Note: Please take the `quantity` column into account!

Optional: Learn how to use `.defaultdict()` to simplify your code.

In [179]:
from collections import defaultdict
chip_dict = defaultdict(int)
for row in raw_data.values:
    item_name = row[2]
    if 'Chips' in item_name:
        quant = row[1]
        chip_dict[item_name] = chip_dict[item_name] + quant 
chip_dict
 

defaultdict(int,
            {'Chips': 230L,
             'Chips and Fresh Tomato Salsa': 130L,
             'Chips and Guacamole': 506L,
             'Chips and Mild Fresh Tomato Salsa': 1L,
             'Chips and Roasted Chili Corn Salsa': 23L,
             'Chips and Roasted Chili-Corn Salsa': 18L,
             'Chips and Tomatillo Green Chili Salsa': 45L,
             'Chips and Tomatillo Red Chili Salsa': 50L,
             'Chips and Tomatillo-Green Chili Salsa': 33L,
             'Chips and Tomatillo-Red Chili Salsa': 25L,
             'Side of Chips': 110L})

---

## Bonus: Craft a problem statement about this data that interests you, and then answer it!


In [ ]:
# Problem Statement
'''
Do they sell more burritos or bowles? I always have bowls so am just a liitle curious..
And what is thier most popular food in general? 
'''

In [188]:
# looks like bowl was slightly more popular than burritos
bowles_sales_in_quantity = sum([row[1] for row in raw_data.values if 'Bowl' in row[2]])
burritos_sales_in_quantity = sum([row[1] for row in raw_data.values if 'Burrito' in row[2]])
print 'bowles_sales_in_quantity:',bowles_sales_in_quantity
print 'burritos_sales_in_quantity:', burritos_sales_in_quantity

bowles_sales_in_quantity: 1398
burritos_sales_in_quantity: 1231


In [201]:
# so Chicken Bowl is the most popular? I am more of a fan of Steak Bowl
raw_data.groupby('item_name')['quantity'].sum().sort_values(ascending=False)

item_name
Chicken Bowl                             761
Chicken Burrito                          591
Chips and Guacamole                      506
Steak Burrito                            386
Canned Soft Drink                        351
Chips                                    230
Steak Bowl                               221
Bottled Water                            211
Chips and Fresh Tomato Salsa             130
Canned Soda                              126
Chicken Salad Bowl                       123
Chicken Soft Tacos                       120
Side of Chips                            110
Veggie Burrito                            97
Barbacoa Burrito                          91
Veggie Bowl                               87
Carnitas Bowl                             71
Barbacoa Bowl                             66
Carnitas Burrito                          60
Steak Soft Tacos                          56
6 Pack Soft Drink                         55
Chips and Tomatillo Red Chili Salsa       50
